In [ ]:
from stochastic_volatility_model import *

import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from datetime import datetime

In [ ]:
df = pd.read_csv('spx-daily-returns.csv')
# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%d-%b-%y')

# Select the rows between 02 january 2008 and april 01 2009
start_date = '2008-01-02'
end_date = '2009-03-31'

mask = (df['date'] >= start_date) & (df['date'] <= end_date)
df = df.loc[mask]

# Calculate log returns and add it as a new column
df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
Y = df["log_returns"].values[1:]

# Create a 1x2 subplot
fig, ax = plt.subplots(1, 2, figsize=(16, 8))

# Plot 'close' on the first subplot
sns.lineplot(x='date', y='close', data=df, ax=ax[0])
ax[0].set_title('Price over time')
ax[0].set_ylabel('Price')

# Plot 'log_returns' on the second subplot
sns.lineplot(x='date', y='log_returns', data=df, ax=ax[1])
ax[1].set_title('Log returns over time')
ax[1].set_ylabel('Log returns')

plt.tight_layout()
plt.show()

T = len(Y)
N = 10000

print('Application du filtre de Storvik')
X, W, W_estimated, alpha_estimated, beta_estimated, mu_estimated = storvik_SIR_stochastic_volatility_model(T,Y,N)
Storvik_hidden_states_estimated = np.sum(W*X,axis=1)

print('Estimation des paramètres du modèle')
W_cst = np.sum(W_estimated[-1,:]*W[-1,:])
alpha_cst = np.sum(alpha_estimated[-1,:]*W[-1,:])
beta_cst = np.sum(beta_estimated[-1,:]*W[-1,:])

print('W = ',W_cst)
print('alpha = ',alpha_cst)
print('beta = ',beta_cst)

print('Application du SIR')
SIR_W,SIR_X = SIR_stochastic_volatility_model(T,Y,N,W_cst,alpha_cst,beta_cst,0)
SIR_hidden_states_estimated = np.sum(SIR_W*SIR_X,axis=1)

In [ ]:
L_t = df["date"].values[1:]

# Conversion des dates en objets datetime
dates = pd.to_datetime(L_t)

# Création de la figure et de l'axe
fig, ax = plt.subplots(figsize=(20, 8))

# Tracé de la courbe
sns.scatterplot(x=dates, y=Storvik_hidden_states_estimated, label='Storvik', color='blue')
sns.lineplot(x=dates, y=SIR_hidden_states_estimated, label='Filtre SIR avec les paramètres estimés du filtre Storvik', color='blue')

# Définir les localisateurs de mois et le formateur de dates
months = mdates.MonthLocator()
months_fmt = mdates.DateFormatter('%b')

# Appliquer les localisateurs et le formateur d'axes des x
ax = plt.gca()
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(months_fmt)
ax.tick_params(axis='x', labelsize=14)  # Augmenter la taille des mois

# Ajouter le texte pour l'année 2008 et 2009
ax.text(pd.Timestamp(start_date), ax.get_ylim()[0], '2008', ha='center', va='bottom',fontweight='bold')
ax.text(pd.Timestamp(end_date[:4]+'-01-02'), ax.get_ylim()[0], '2009', ha='center', va='bottom',fontweight='bold')

# Conversion des chaînes de caractères en objet datetime
start_date_formatted = datetime.strptime(start_date, '%Y-%m-%d')
end_date_formatted = datetime.strptime(end_date, '%Y-%m-%d')

# Transformation en format plus développé
start_date_long = start_date_formatted.strftime('%B %Y')
end_date_long = end_date_formatted.strftime('%B %Y')

# Titre et légende
plt.xlabel('Temps')
plt.title("Storvik puis SIR sur le S&P500 entre "+start_date_long +" et "+end_date_long+" avec "+str(N)+' particules', fontsize=30, fontname='Times New Roman')
plt.legend()

# Affichage du graphique
plt.show()

In [ ]:
W_parametre = W_estimated
alpha = alpha_estimated
beta = beta_estimated

data_mean = [np.sum(alpha*W, axis=1),np.sum(beta*W, axis=1),np.sum(W_parametre*W, axis=1)]

# Calcul des intervalles de confiance
intervalle_confiance_alpha = np.percentile(alpha, [2.5, 97.5], axis=1)
intervalle_confiance_beta = np.percentile(beta, [2.5, 97.5], axis=1)
intervalle_confiance_W = np.percentile(W_parametre, [2.5, 97.5], axis=1)

# Ajout des intervalles de confiance aux données
data_max = [intervalle_confiance_alpha[1], intervalle_confiance_beta[1], intervalle_confiance_W[1]]
data_min = [intervalle_confiance_alpha[0], intervalle_confiance_beta[0], intervalle_confiance_W[0]]


hist_data = [alpha[-1,:],beta[-1,:],W_parametre[-1,:]]

labels = ['$\\alpha$','$\\beta$','$W$']

# Création de la figure et des sous-graphiques avec GridSpec
fig = plt.figure(tight_layout=True,figsize=(20,9))
gs = gridspec.GridSpec(2, 3, height_ratios=[1, 1])

# Plots pour la ligne 2
for i in range(3):
    ax2 = fig.add_subplot(gs[0, i])
    sns.lineplot(x=L_t,y=data_mean[i],ax=ax2)
    ax2.fill_between(L_t, data_max[i], data_min[i],color='grey',alpha=0.3,label='intervalle de confiance à 95%')
    ax2.set_title(labels[i])
    ax2.set_xlabel('Temps')
    ax2.legend()

# Plots pour la ligne 3
# Last filter step 
for i in range(3):
    ax3 = fig.add_subplot(gs[1, i])
    sns.histplot(hist_data[i], ax=ax3)
    ax3.set_title(labels[i])
    ax3.set_xlabel('Valeur de '+ str(labels[i]))

# Ajustement de l'espacement entre les sous-graphiques
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Ajustez les valeurs ([left, bottom, right, top]) pour définir l'espacement souhaité
fig.suptitle("Estimated parameters with Storvik's filter on S&P500 between Jan. 2008 and Mar. 2009 with "+str(N)+' particles', fontsize=30,fontname='Times New Roman')  

# Affichage de la figure
plt.show()